In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
words_to_index = {'hello': 0, 'world': 1}
embeddings = nn.Embedding(2, 5)
print (embeddings.type)
lookup_tensor = torch.LongTensor([words_to_index["hello"]])
print (lookup_tensor.shape)
hello_embed = embeddings(autograd.Variable(lookup_tensor))
print (hello_embed)

In [ ]:
CONTEXT_SIZE=2
EMBEDDING_DIM=10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
print (len(test_sentence))

trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2)]
print (trigrams[:3])
vocab = set(test_sentence)
print (vocab)
word_to_index = {word: i for i, word in enumerate(vocab)}
print (word_to_index.keys())

In [ ]:
class LanguageModelNN(nn.Module):
    def __init__(self, vocab_size, context_size, embedding_dim):
        super(LanguageModelNN, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        # print (inputs.type)
        embeds = self.embeddings(inputs).view(1, -1)
        # print (embeds.type)
        linear1 = F.relu(self.linear1(embeds))
        linear2 = self.linear2(linear1)
        return F.log_softmax(linear2)

losses = []
loss_function = nn.NLLLoss()
model = LanguageModelNN(len(vocab), CONTEXT_SIZE, EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001)

def make_context_vector(context, word_to_index):
    context_ids = [word_to_index[i] for i in context]
    # print (context_ids)
    return autograd.Variable(torch.LongTensor(context_ids))

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_ids = make_context_vector(context, word_to_index)
        # print (conetxt_ids)
        model.zero_grad()
        output = model(context_ids)
        # print (output)
        loss = loss_function(output, make_context_vector([target], word_to_index))
        
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
    
print (losses)